데이터 불러오기

In [ ]:
import pandas as pd

games = pd.read_csv("Data/games.csv", parse_dates=["GAME_DATE_EST"])
details = pd.read_csv("Data/games_details.csv", low_memory=False)
teams = pd.read_csv("Data/teams.csv")
ranking = pd.read_csv("Data/ranking.csv")
player = pd.read_csv("Data/players.csv")

배경 지식 : 

분석관점 : 우승 시즌을 기준으로, 팀의 핵심 포지션 선수들이 리그 전체 기준에서 얼마나 경쟁력을 갖추게 되었는가?

내가 필요로 하는 데이터 및 컬럼
games.csv
'GAME_ID", "SEASON"

games_details.csv
'FG_PCT', 'FG3_PCT', 'FT_PCT', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PTS', 'PLUS_MINUS', 'FGM', 'FGA', 'FG3M', 'FTA'

games 전처리

In [ ]:
games.info()
games.isnull().sum()

In [ ]:
games['SEASON']

99개의 정체 확인

In [ ]:
error_games = games[games['PTS_home'].isnull()]

print(error_games['GAME_STATUS_TEXT'].value_counts())

상위 5개 행만 확인

In [ ]:
print(error_games[['GAME_DATE_EST', 'GAME_STATUS_TEXT', 'HOME_TEAM_ID']].head())

경기가 끝난상태인데 기록이 없는 것이므로 삭제 진행.

In [ ]:
clean_games = games.dropna(subset = ["PTS_home"])

print(clean_games.info())

games_details 전처리

In [ ]:
details.info()

In [ ]:
details.isnull().sum()

결측치1 : 109690건의 데이터부터 전처리 시작

MIN 출전시간이 없는 선수 왜 그런지 확인

In [ ]:
error_min = details[details['MIN'].isnull()]

print(error_min['COMMENT'].value_counts().head())

혹시나 점수 기록이 있는지 확인해보았다

In [ ]:
print(error_min['PTS'].describe())

기록이 없는 선수들이므로 삭제(분석 관점이 선수들의 평균 기록 비교이기 때문)

In [ ]:
clean_details = details.dropna(subset = 'MIN')

결측치2 : START_POSITION이 없는 41만건 확인,
벤치에 있는 선수여서 선발 포지션이 없는지 확인

In [ ]:
if_bench = details[(details['START_POSITION'].isnull())]

if_bench = details[(details['MIN'].notnull())]

if_bench.head()

벤치 선수의 출전 기록 확인 후

선발은 양팀 합쳐서 10명인데 10이 아닌 경기도 있을 수 있음

In [ ]:
starters_count = clean_details.groupby("GAME_ID")["START_POSITION"].count()

bad_games = starters_count[starters_count != 10].index

print(len(starters_count))
print(len(bad_games))

print(1024/26523*100)

중복데이터이거나, 입력 누락, 선발이 0의 경우 아무 기록도 없기 때문에 삭제가 가능하다고 판단
평균에 큰 영향을 안끼칠거 같아 1024개 삭제 진행

In [ ]:
clean_details = clean_details[~clean_details['GAME_ID'].isin(bad_games)].copy()

선발은 아니었지만 경기기록이 있는 선수는 벤치로 분류

In [ ]:
clean_details['START_POSITION'] = clean_details['START_POSITION'].fillna('bench')

In [ ]:
clean_details.head(20)

In [ ]:
clean_details.info()

In [ ]:
print(clean_games['SEASON'].unique())

클린 데이터 merge 진행

clean_games 에서 시즌 정보를 가져와야함

GAME_ID를 기준으로 Left 진행

In [ ]:
merged_df = pd.merge(clean_details, clean_games[['GAME_ID', 'SEASON']], on = 'GAME_ID', how = 'left')

merged_df.info()

전처리

MIN 출전시간이 문자로 되어 있어 숫자로 변환해야함

숫자로 만드는 함수 먼저 진행

파생컬럼 'PLAY_TIME' 생성

In [ ]:
def min_to_float(x) :
    try :
        if isinstance(x, str) and ':' in x :
            m, s = x.split(':')
            return float(m) + float(s)/60
        else :
            return float(x)
    except :
        return 0.0

merged_df['PLAY_TIME'] = merged_df['MIN'].apply(min_to_float)

merged_df.head(10)

14-15 시즌 정보를 필요로 하기 때문에
시즌 전체와 팀 정보 분류
(별칭은 팀에서 정한 기준으로 작성 14-15 시즌이면 15로 지정)

In [ ]:
nba_15 = merged_df[merged_df['SEASON'] == 2014].copy()

gsw_15 = nba_15[nba_15['TEAM_ABBREVIATION'] == 'GSW'].copy()

gsw_15.head(10)

궁금증 발생!

선발 출장 수 / 벤치 출장 수
    선발 시 어떤 포지션으로 출전 했는지? - 시간 남으면 하자..

출장 경기 수

우리팀만 분류하기

In [ ]:
nba_15['is_start'] = nba_15['START_POSITION'] != 'bench'
nba_15['is_bench'] = nba_15['START_POSITION'] == 'bench'

start_or_bench = nba_15.groupby(['PLAYER_NAME', 'TEAM_ABBREVIATION'])[['is_start', 'is_bench']].sum().reset_index()

start_or_bench['total_play_year'] = start_or_bench['is_start'] + start_or_bench['is_bench']

print(start_or_bench[start_or_bench['TEAM_ABBREVIATION'] == 'GSW'].sort_values('is_start', ascending = False))

In [ ]:
gsw_roster = start_or_bench[start_or_bench['TEAM_ABBREVIATION'] == 'GSW'].sort_values(by=['is_start', 'total_play_year'], ascending=[False, False])

import matplotlib.pyplot as plt

plt.figure(figsize = (16, 7))

players = gsw_roster['PLAYER_NAME']
starts = gsw_roster['is_start']
bench = gsw_roster['is_bench']

gsw_blue = '#1D428A'
gsw_yellow = '#FFC72C'
gsw_slate = '#26282A'

p1 = plt.bar(players, starts, label='Starter (선발)', color=gsw_blue)
p2 = plt.bar(players, bench, bottom=starts, label='Bench (벤치)', color=gsw_yellow)

plt.title('2014-15 GSW Roster Analysis: Starter vs Bench', fontsize=16, fontweight='bold')
plt.ylabel('Games Played', fontsize=12)
plt.xlabel('Name', fontsize=12)
plt.xticks(rotation=90, ha='right')
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=1)

plt.tight_layout()
plt.show()

In [ ]:
merged_df.to_csv('clean_data.csv', index=False, encoding='utf-8-sig')

팀 주전 선수와 리그 상위선수들, 평균과의 비교
    - 공격기준, 수비기준

1차 지표들을 통해 2차 지표 생성
    
    TS% = PTS / 2(FGA + 0.44 * FTA)
    0.44는 테크니컬 파울이나 앤드원(And-1) 상황을 고려하여 도출된 통계적 계수

    eFG% = FGM + (0.5 * 3PM) / FGA
    3점슛에 가중치 0.5를 줌
    
    Net Rating : 100 * (OFFRtg - DEFRtg)
        +10 이상이면 리그를 지배하는 수준, 만약 개인기록은 평범하지만 Net Rating이 높다면 보이지 않는 부분에서 팀 승리에 결정적인 역할을 함

In [ ]:
df = pd.read_csv('/Users/t2025-m0050/Introductory-Project/clean_data.csv')

팀의 주요 선수들이 리그 주요 선수들과 비교했을떄 경쟁력을 가지는지를 확인

In [ ]:
df_2014 = df[(df['SEASON'] == 2014) & (df['START_POSITION'].isin(['G', 'F', 'C']))]

basic_stats = ['FG_PCT', 'FG3_PCT', 'FT_PCT', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PTS', 'PLUS_MINUS', 'FGM', 'FGA', 'FG3M', 'FTA']

player_avg = df_2014.groupby(['PLAYER_NAME', 'TEAM_ABBREVIATION', 'START_POSITION'])[basic_stats].mean(numeric_only=True).reset_index()

game_counts = df_2014.groupby(['PLAYER_NAME', 'TEAM_ABBREVIATION', 'START_POSITION'])['GAME_ID'].count().reset_index(name='GAME_COUNT')
player_avg = pd.merge(player_avg, game_counts, on=['PLAYER_NAME', 'TEAM_ABBREVIATION', 'START_POSITION'])
player_avg = player_avg[player_avg['GAME_COUNT'] >= 10]

player_avg['eFG%'] = (player_avg['FGM'] + 0.5 * player_avg['FG3M']) / player_avg['FGA'] * 100

player_avg['TS%'] = player_avg['PTS'] / (2 * (player_avg['FGA'] + 0.44 * player_avg['FTA'])) * 100

gsw_players = player_avg[player_avg['TEAM_ABBREVIATION'] == 'GSW']
league_players = player_avg[player_avg['TEAM_ABBREVIATION'] != 'GSW']

gsw_g = gsw_players[gsw_players['START_POSITION'] == 'G'].sort_values('PTS', ascending=False).head(2)
gsw_f = gsw_players[gsw_players['START_POSITION'] == 'F'].sort_values('PTS', ascending=False).head(2)
gsw_c = gsw_players[gsw_players['START_POSITION'] == 'C'].sort_values('PTS', ascending=False).head(1)

top_g = league_players[league_players['START_POSITION'] == 'G'].sort_values('PTS', ascending=False).head(10)
top_f = league_players[league_players['START_POSITION'] == 'F'].sort_values('PTS', ascending=False).head(10)
top_c = league_players[league_players['START_POSITION'] == 'C'].sort_values('PTS', ascending=False).head(5)

gsw_starters = pd.concat([gsw_g, gsw_f, gsw_c])
league_top = pd.concat([top_g, top_f, top_c])

print("GSW 주전 경쟁력")
print(gsw_starters.groupby('START_POSITION').mean(numeric_only=True).round(2))

print("리그 상위권 평균")
print(league_top.groupby('START_POSITION').mean(numeric_only=True).round(2))

In [ ]:
df_2014_sub = df[(df['SEASON'] == 2014) & (df['START_POSITION'].isin(['G', 'F', 'C']))]

basic_stats = ['FG_PCT', 'FG3_PCT', 'FT_PCT', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PTS', 'PLUS_MINUS', 'FGM', 'FGA', 'FG3M', 'FTA']

player_avg = df_2014.groupby(['PLAYER_NAME', 'TEAM_ABBREVIATION', 'START_POSITION'])[basic_stats].mean(numeric_only=True).reset_index()

game_counts = df_2014.groupby(['PLAYER_NAME', 'TEAM_ABBREVIATION', 'START_POSITION'])['GAME_ID'].count().reset_index(name='GAME_COUNT')
player_avg = pd.merge(player_avg, game_counts, on=['PLAYER_NAME', 'TEAM_ABBREVIATION', 'START_POSITION'])
player_avg = player_avg[player_avg['GAME_COUNT'] >= 10]

player_avg['eFG%'] = (player_avg['FGM'] + 0.5 * player_avg['FG3M']) / player_avg['FGA'] * 100

player_avg['TS%'] = player_avg['PTS'] / (2 * (player_avg['FGA'] + 0.44 * player_avg['FTA'])) * 100

league_2014_players = player_avg

avg_g = league_2014_players[league_2014_players['START_POSITION'] == 'G'].sort_values('PTS', ascending=False).head(2)
avg_f = league_2014_players[league_2014_players['START_POSITION'] == 'F'].sort_values('PTS', ascending=False).head(2)
avg_c = league_2014_players[league_2014_players['START_POSITION'] == 'C'].sort_values('PTS', ascending=False).head(1)

league_avg = pd.concat([avg_g, avg_f, avg_c])

print("리그 선발 평균")
print(league_avg.groupby('START_POSITION').mean(numeric_only=True).round(2))

In [ ]:
Off_cols = ['PTS', 'AST', 'eFG%', 'TS%']
Def_cols = ['REB', 'STL', 'BLK', 'TO']

print("GSW 주전 공격 경쟁력")
print(gsw_starters.groupby('START_POSITION')[Off_cols].mean(numeric_only=True).round(2))

print("리그 상위권 공격지표 평균")
print(league_top.groupby('START_POSITION')[Off_cols].mean(numeric_only=True).round(2))

In [ ]:
print("GSW 주전 수비 경쟁력")
print(gsw_starters.groupby('START_POSITION')[Def_cols].mean(numeric_only=True).round(2))

print("리그 상위권 수비지표 평균")
print(league_top.groupby('START_POSITION')[Def_cols].mean(numeric_only=True).round(2))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

gsw_mean = gsw_starters.groupby('START_POSITION').mean(numeric_only=True)
league_mean = league_top.groupby('START_POSITION').mean(numeric_only=True)

def plot_comparison(gsw_data, league_data, columns, title):
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    positions = ['G', 'F', 'C']
    
    gsw_blue = '#1D428A'
    gsw_yellow = '#FFC72C'
    gsw_slate = '#26282A'
    
    axes = axes.flatten()
    
    for i, col in enumerate(columns):
        ax = axes[i]
        
        gsw_vals = [gsw_data.loc[pos, col] for pos in positions]
        league_vals = [league_data.loc[pos, col] for pos in positions]
        
        x = np.arange(len(positions))
        width = 0.35
        
        rects1 = ax.bar(x - width/2, gsw_vals, width, label='GSW', color=gsw_blue)
        rects2 = ax.bar(x + width/2, league_vals, width, label='League Top', color=gsw_yellow)
        
        ax.set_title(col, fontsize=14, fontweight='bold')
        ax.set_xticks(x)
        ax.set_xticklabels(positions, fontsize=12)
        ax.grid(axis='y', linestyle='--', alpha=0.3)
        
        ax.bar_label(rects1, padding=3, fmt='%.1f', fontsize=9)
        ax.bar_label(rects2, padding=3, fmt='%.1f', fontsize=9)
        
        if i == 0:
            ax.legend()
            
    fig.suptitle(title, fontsize=18, y=1.05)
    plt.tight_layout()
    plt.show()

Off_cols = ['PTS', 'AST', 'eFG%', 'TS%']
plot_comparison(gsw_mean, league_mean, Off_cols, 'GSW vs League Elite: Offense')

Def_cols = ['REB', 'STL', 'BLK', 'TO']
plot_comparison(gsw_mean, league_mean, Def_cols, 'GSW vs League Elite: Defense')

In [ ]:
df = pd.read_csv('Data/clean_data.csv')

df_2014 = df[df['SEASON'] == 2014]

bench_df = df_2014[df_2014['START_POSITION'] == 'bench']

sub_avg = bench_df.groupby(['PLAYER_NAME', 'TEAM_ABBREVIATION'])[basic_stats].mean(numeric_only=True).reset_index()

bench_counts = bench_df.groupby(['PLAYER_NAME', 'TEAM_ABBREVIATION'])['GAME_ID'].count().reset_index(name='GAME_COUNT')
sub_avg = pd.merge(sub_avg, bench_counts, on=['PLAYER_NAME', 'TEAM_ABBREVIATION'])

sub_avg = sub_avg[sub_avg['GAME_COUNT'] >= 30]

sub_avg['eFG%'] = (sub_avg['FGM'] + 0.5 * sub_avg['FG3M']) / sub_avg['FGA'] * 100
sub_avg['TS%'] = sub_avg['PTS'] / (2 * (sub_avg['FGA'] + 0.44 * sub_avg['FTA'])) * 100

gsw_sub = sub_avg[sub_avg['TEAM_ABBREVIATION'] == 'GSW']
league_sub = sub_avg[sub_avg['TEAM_ABBREVIATION'] != 'GSW']

cols = ['PTS', 'AST', 'REB', 'TS%', 'eFG%', 'PLUS_MINUS']

print(f"GSW 벤치 평균 (대상: {len(gsw_sub)}명)")
print(gsw_sub[cols].mean(numeric_only=True).round(2))

print(f"리그 전체 벤치 평균 (대상: {len(league_sub)}명)")
print(league_sub[cols].mean(numeric_only=True).round(2))